In [13]:
import pandas as pd
from os import mkdir

# Tratamento da Base de Dados

Antes de começar, faça o download dos dados originais, descompacte-os e os coloque numa pasta chamada "dados-originais" no mesmo diretório desse arquivo main.
https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads

## Mapeamento e tradução

Toda a base de dados usa os dados em um formato de números para representar possíveis respostas, então é preciso fazer um mapeamento dos dados desses valores numéricos para o que eles realmente significam.

As colunas estão descritas em forma de código do questionário usado, exemplos: 'A001A', 'B00113', 'F001'. Então aqui traduziremos as colunas para que fiquem de uma forma mais legível. Para isso temos a função `traduzir_colunas()`

### Mapeamento dos Dados e Dicionário das Colunas

In [14]:
mapeamento_situacao_domicilio = {
    1: 'Urbana',
	2: 'Rural'
}

mapeamento_condicao_domicilio = {
    1: 'Pessoa responsável pelo domicílio',
    2: 'Cônjuge ou companheiro(a) de sexo diferente',
    3: 'Cônjuge ou companheiro(a) do mesmo sexo',
    4: 'Filho(a) do responsável e do cônjuge',
    5: 'Filho(a) somente do responsável',
    6: 'Filho(a) somente do cônjuge',
    7: 'Genro ou nora',
    8: 'Pai, mãe, padrasto ou madrasta',
    9: 'Sogro(a)',
    10: 'Neto(a)',
    11: 'Bisneto(a)',
    12: 'Irmão ou irmã',
    13: 'Avô ou avó',
    14: 'Outro parente',
    15: 'Agregado(a) - Não parente que não compartilha despesas',
    16: 'Convivente - Não parente que compartilha despesas',
    17: 'Pensionista',
    18: 'Empregado(a) doméstico(a)',
    19: 'Parente do(a) empregado(a) doméstico(a)'
}

mapeamento_sexo = {
    1: 'Homem',
	2: 'Mulher'
}

mapeamento_cor_raca = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena',
    9: 'Ignorado'
}

mapeamento_escolaridade = {
    1: 'Sem instrução',
    2: 'Fundamental incompleto',
    3: 'Fundamental completa',
    4: 'Médio incompleto',
    5: 'Médio completo',
    6: 'Superior incompleto',
    7: 'Superior completo',
    8: 'Pós-graduação, mestrado ou doutorado'
}

mapeamento_frequenta_escola = {
    1: 'Sim',
    2: 'Não'
}

mapeamento_escola_publica_privada = {
    1: 'Pública',
    2: 'Privada'
}

mapeamento_aulas_presenciais = {
    1: 'Sim, normalmente',
    2: 'Sim, mas apenas parcialmente',
    3: 'Não, e meu normalmente é presencial/semipresencial',
    4: 'Não, meu curso é online'
}

mapeamento_sintomas = {
    1: 'Sim',
    2: 'Não',
    3: 'Não sabe',
    9: 'Ignorado'
}

mapeamento_teste_covid = {
    1: 'Sim',
    2: 'Não',
    9: 'Ignorado'
}

mapeamento_resultado_teste = {
    1: 'Positivo',
    2: 'Negativo',
    3: 'Inconclusivo',
    4: 'Ainda não recebeu o resultado',
    9: 'Ignorado'
}

mapeamento_tipo_trabalho = {
    1: 'Trabalhador doméstico (empregado doméstico, cuidados, babá)',
    2: 'Militar do exército, marinha ou aeronáutica',
    3: 'Policial militar ou bombeiro militar',
    4: 'Empregado do setor privado',
    5: 'Empregado do setor público (inclusive empresas de economia mista)',
    6: 'Empregador',
    7: 'Conta própria',
    8: 'Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente',
    9: 'Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)'
}

mapeamento_area_trabalho = {
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal'
}

mapeamento_carteira_trabalho = {
    1: 'Sim, tem carteira de trabalho assinada',
    2: 'Sim, é servidor público estatutário',
    3: 'Não'
}

mapeamento_tipo_cargo = {
    1: 'Empregado doméstico, diarista, cozinheiro (em domicílios particulares)',
    2: 'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada)',
    3: 'Auxiliar de escritório, escriturário',
    4: 'Secretária, recepcionista',
    5: 'Operador de Telemarketing',
    6: 'Comerciante (dono do bar, da loja etc.)',
    7: 'Balconista, vendedor de loja',
    8: 'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)',
    9: 'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)',
    10: 'Cozinheiro e garçom (de restaurantes, empresas)',
    11: 'Padeiro, açougueiro e doceiro',
    12: 'Agricultor, criador de animais, pescador, silvicultor e jardineiro',
    13: 'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)',
    14: 'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)',
    15: 'Motorista de caminhão (caminhoneiro)',
    16: 'Motoboy',
    17: 'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.)',
    18: 'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro',
    19: 'Mecânico de veículos, máquinas industriais etc.',
    20: 'Artesão, costureiro e sapateiro',
    21: 'Cabeleireiro, manicure e afins',
    22: 'Operador de máquinas, montador na indústria',
    23: 'Auxiliar de produção, de carga e descarga',
    24: 'Professor da educação infantil, de ensino fundamental, médio ou superior',
    25: 'Pedagogo, professor de idiomas, música, arte e reforço escolar',
    26: 'Médico, enfermeiro, profissionais de saúde de nível superior',
    27: 'Técnico, profissional da saúde de nível médio',
    28: 'Cuidador de crianças, doentes ou idosos',
    29: 'Segurança, vigilante, outro trabalhador dos serviços de proteção',
    30: 'Policial civil',
    31: 'Porteiro, zelador',
    32: 'Artista, religioso (padre, pastor etc.)',
    33: 'Diretor, gerente, cargo político ou comissionado',
    34: 'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)',
    35: 'Outro técnico ou profissional de nível médio',
    36: 'Outros'
}

mapeamento_atividade_empresa = {
    1: 'Agricultura, pecuária, produção florestal e pesca',
    2: 'Extração de petróleo, carvão mineral, minerais metálicos, pedra, areia, sal etc.',
    3: 'Indústria da transformação (inclusive confecção e fabricação caseira)',
    4: 'Fornecimento de eletricidade e gás, água, esgoto e coleta de lixo',
    5: 'Construção',
    6: 'Comércio no atacado e varejo',
    7: 'Reparação de veículos automotores e motocicletas',
    8: 'Transporte de passageiros',
    9: 'Transporte de mercadorias',
    10: 'Armazenamento, correios e serviços de entregas',
    11: 'Hospedagem (hotéis, pousadas etc.)',
    12: 'Serviço de alimentação (bares, restaurantes, ambulantes de alimentação)',
    13: 'Informação e comunicação (jornais, rádio e televisão, telecomunicações e informática)',
    14: 'Bancos, atividades financeiras e de seguros',
    15: 'Atividades imobiliárias',
    16: 'Escritórios de advocacia, engenharia, publicidade e veterinária (Atividades profissionais, científicas e técnicas)',
    17: 'Atividades de locação de mão de obra, segurança, limpeza, paisagismo e teleatendimento',
    18: 'Administração pública (governo federal, estadual e municipal)',
    19: 'Educação',
    20: 'Saúde humana e assistência social',
    21: 'Organizações religiosas, sindicatos e associações',
    22: 'Atividade artísticas, esportivas e de recreação',
    23: 'Cabeleireiros, tratamento de beleza e serviços pessoais',
    24: 'Serviço doméstico remunerado (será imputado da posição na ocupação)',
    25: 'Outro'
}

mapeamento_faixa_rendimento = {
    0: '0 - 100',
    1: '101 - 300',
    2: '301 - 600',
    3: '601 - 800',
    4: '801 - 1.600',
    5: '1.601 - 3.000',
    6: '3.001 - 10.000',
    7: '10.001 - 50.000',
    8: '50.001 - 100.000',
    9: 'Mais de 100.000'
}

mapeamento_tipo_domicilio = {
    1: 'Próprio - já pago',
    2: 'Próprio - ainda pagando',
    3: 'Alugado',
    4: 'Cedido por empregador',
    5: 'Cedido por familiar',
    6: 'Cedido de outra forma',
    7: 'Outra condição'
}

dicionario_colunas = {
    'A002': 'Idade',
    'A003': 'Sexo',
    'A004': 'Cor',
    'A005': 'Escolaridade',
    'A006': 'Frequenta_escola',
    'A006A': 'Escola_eh',
    'A006B': 'Aulas_presenciais',
    'B0011': 'Febre',
    'B0012': 'Tosse',
    'B0013': 'Dor_garganta',
    'B0014': 'Dificuldade_respirar',
    'B0015': 'Dor_cabeca',
    'B0016': 'Dor_peito',
    'B0017': 'Nausea',
    'B0018': 'Nariz_entupido',
    'B0019': 'Fadiga',
    'B00110': 'Dor_olhos',
    'B00111': 'Perda_cheiro',
    'B00112': 'Dor_muscular',
    'B00113': 'Diarreia',
    'B008': 'Teste_coronavírus',
    'B009A': 'Exame SWAB',
    'B009B': 'Resultado_SWAB',
    'B009C': 'Exame_sangue_dedo',
    'B009D': 'Resultado_sangue_dedo',
    'B009E': 'Exame_sangue_braco',
    'B009F': 'Resultado_sangue_braco',
    'C007': 'Trabalho',
    'C007A': 'Area_trabalho',
    'C01011': 'Faixa_renda'
}

### Funções

In [15]:
def mapear(df, coluna, dicionario_map):
    if coluna in df.columns:
        df[coluna] = df[coluna].map(dicionario_map)

def traduzir_dados(df: pd.DataFrame):
    # Aplicar as traduções para cada coluna
    mapear(df, 'V1022', mapeamento_situacao_domicilio)
    mapear(df, 'A001A', mapeamento_condicao_domicilio)
    mapear(df, 'A003', mapeamento_sexo)
    mapear(df, 'A004', mapeamento_cor_raca)
    mapear(df, 'A005', mapeamento_escolaridade)
    mapear(df, 'A006', mapeamento_frequenta_escola)
    mapear(df, 'A006A', mapeamento_escola_publica_privada)
    mapear(df, 'A006B', mapeamento_aulas_presenciais)
    
    # Traduzir sintomas e testes COVID-19
    colunas_sintomas = [
        'B0011', 'B0012', 'B0013', 'B0014', 'B0015', 'B0016', 'B0017',
        'B0018', 'B0019', 'B00110', 'B00111', 'B00112', 'B00113' 
    ]
    
    for coluna in colunas_sintomas:
        mapear(df, coluna, mapeamento_sintomas)

    colunas_testes = [
        'B008', 'B009A', 'B009C', 'B009E'
    ]
    for coluna in colunas_testes:
        mapear(df, coluna, mapeamento_teste_covid)

    colunas_resultados = [
        'B009B', 'B009D', 'B009F'
    ]
    
    for coluna in colunas_resultados:
        mapear(df, coluna, mapeamento_resultado_teste)

    # Traduzir informações de trabalho
    mapear(df, 'C007', mapeamento_tipo_trabalho)
    mapear(df, 'C007A', mapeamento_area_trabalho)
    mapear(df, 'C007B', mapeamento_carteira_trabalho)
    mapear(df, 'C007C', mapeamento_tipo_cargo)
    mapear(df, 'C007D', mapeamento_atividade_empresa)

    # Traduzir rendimento e domicílio
    mapear(df, 'C01011', mapeamento_faixa_rendimento)
    mapear(df, 'F001', mapeamento_tipo_domicilio)

def traduzir_colunas(df):
    for coluna in df.columns:
        if coluna in dicionario_colunas:
            df.rename(columns={coluna: dicionario_colunas[coluna]}, inplace=True)

# Tratamento dos dados

In [16]:
def filtrar_paraiba(df):
    df_paraiba = df.copy()
    df_paraiba = df[df['UF'] == 25]
    return df_paraiba

def filtrar_colunas_desejadas(df: pd.DataFrame):
    colunas_desejadas = list(dicionario_colunas.keys())
    
    colunas_existentes = [coluna for coluna in colunas_desejadas if coluna in df.columns]
    
    return df[colunas_existentes]

def tratamento_dados(df):
    df = filtrar_paraiba(df)
    df = filtrar_colunas_desejadas(df)
    traduzir_dados(df)
    traduzir_colunas(df)
    return df

In [17]:
try:
    mkdir('dados-tratados')
    print('Pasta "dados-tratados" criado com sucesso.')
except FileExistsError:
    print('Pasta "dados-tratados" já existe.')

for i in range(5, 12):
    arquivo = f'dados-originais/PNAD_COVID_{i:0>2}2020.csv'
    
    try:
        df = pd.read_csv(arquivo)
    except FileNotFoundError:
        print(f'Falha ao ler o arquivo "{arquivo}".')
        continue

    df = tratamento_dados(df)

    df.to_csv(f'dados-tratados/pc-{i:0>2}.csv',sep = ";")
    
    display(df)

Pasta "dados-tratados" já existe.


,Idade,Sexo,Cor,Escolaridade,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,Dor_peito,Nausea,Nariz_entupido,Fadiga,Dor_olhos,Perda_cheiro,Dor_muscular,Trabalho,Area_trabalho,Faixa_renda
87619,43,Mulher,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Conta própria,NaN,101 - 300
87620,18,Mulher,Parda,Médio incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
87621,45,Mulher,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor privado,NaN,801 - 1.600
87622,58,Homem,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor privado,NaN,1.601 - 3.000
87623,8,Homem,Parda,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96273,73,Mulher,Branca,Fundamental incompleto,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não,Sim,Não,NaN,NaN,NaN
96274,50,Homem,Parda,Sem instrução,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
96275,34,Mulher,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
96276,36,Homem,Parda,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor público (inclusive empresas...,Municipal,801 - 1.600


,Idade,Sexo,Cor,Escolaridade,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,Dor_peito,Nausea,Nariz_entupido,Fadiga,Dor_olhos,Perda_cheiro,Dor_muscular,Trabalho,Area_trabalho,Faixa_renda
98062,43,Mulher,Parda,Médio completo,Não,Não,Não,Não,Sim,Não,Sim,Não,Sim,Não,Sim,Sim,Conta própria,NaN,101 - 300
98063,18,Mulher,Parda,Médio incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
98064,45,Mulher,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor privado,NaN,801 - 1.600
98065,58,Homem,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor privado,NaN,1.601 - 3.000
98066,8,Homem,Parda,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107696,73,Mulher,Branca,Fundamental incompleto,Não,Sim,Não,Não,Não,Sim,Não,Sim,Não,Sim,Não,Sim,NaN,NaN,NaN
107697,50,Homem,Parda,Sem instrução,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,NaN,NaN,NaN
107698,34,Mulher,Parda,Médio completo,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
107699,36,Homem,Parda,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Empregado do setor público (inclusive empresas...,Municipal,801 - 1.600


,Idade,Sexo,Cor,Escolaridade,Frequenta_escola,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,...,Teste_coronavírus,Exame SWAB,Resultado_SWAB,Exame_sangue_dedo,Resultado_sangue_dedo,Exame_sangue_braco,Resultado_sangue_braco,Trabalho,Area_trabalho,Faixa_renda
100074,43,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100075,18,Mulher,Parda,Médio incompleto,Sim,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100076,46,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor privado,NaN,801 - 1.600
100077,58,Homem,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor privado,NaN,1.601 - 3.000
100078,9,Homem,Parda,Fundamental incompleto,Sim,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109927,73,Mulher,Branca,Fundamental incompleto,NaN,Não,Sim,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109928,50,Homem,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109929,35,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
109930,36,Homem,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,1.601 - 3.000


,Idade,Sexo,Cor,Escolaridade,Frequenta_escola,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,...,Teste_coronavírus,Exame SWAB,Resultado_SWAB,Exame_sangue_dedo,Resultado_sangue_dedo,Exame_sangue_braco,Resultado_sangue_braco,Trabalho,Area_trabalho,Faixa_renda
102116,43,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Conta própria,NaN,101 - 300
102117,18,Mulher,Parda,Médio incompleto,Sim,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102118,58,Homem,Preta,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Inconclusivo,Não,NaN,Conta própria,NaN,801 - 1.600
102119,51,Mulher,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
102120,83,Mulher,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112035,73,Mulher,Branca,Fundamental incompleto,NaN,Não,Sim,Sim,Não,Sim,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112036,50,Homem,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112037,35,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
112038,36,Homem,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,1.601 - 3.000


,Idade,Sexo,Cor,Escolaridade,Frequenta_escola,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,...,Teste_coronavírus,Exame SWAB,Resultado_SWAB,Exame_sangue_dedo,Resultado_sangue_dedo,Exame_sangue_braco,Resultado_sangue_braco,Trabalho,Area_trabalho,Faixa_renda
102395,43,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Conta própria,NaN,101 - 300
102396,18,Mulher,Parda,Médio incompleto,Sim,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102397,58,Homem,Preta,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,Conta própria,NaN,801 - 1.600
102398,51,Mulher,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
102399,83,Mulher,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112436,73,Mulher,Branca,Fundamental incompleto,NaN,Sim,Não,Não,Não,Sim,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112437,50,Homem,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112438,35,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
112439,36,Homem,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,1.601 - 3.000


,Idade,Sexo,Cor,Escolaridade,Frequenta_escola,Febre,Tosse,Dor_garganta,Dificuldade_respirar,Dor_cabeca,...,Teste_coronavírus,Exame SWAB,Resultado_SWAB,Exame_sangue_dedo,Resultado_sangue_dedo,Exame_sangue_braco,Resultado_sangue_braco,Trabalho,Area_trabalho,Faixa_renda
100066,43,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Conta própria,NaN,101 - 300
100067,18,Mulher,Parda,Médio incompleto,Sim,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100068,58,Homem,Preta,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,Conta própria,NaN,801 - 1.600
100069,47,Mulher,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
100070,83,Mulher,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109829,74,Mulher,Branca,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
109830,50,Homem,Parda,Sem instrução,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109831,35,Mulher,Parda,Médio completo,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
109832,36,Homem,Parda,Fundamental incompleto,NaN,Não,Não,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,1.601 - 3.000


,Idade,Sexo,Cor,Escolaridade,Frequenta_escola,Escola_eh,Aulas_presenciais,Febre,Tosse,Dor_garganta,...,Teste_coronavírus,Exame SWAB,Resultado_SWAB,Exame_sangue_dedo,Resultado_sangue_dedo,Exame_sangue_braco,Resultado_sangue_braco,Trabalho,Area_trabalho,Faixa_renda
100336,43,Mulher,Parda,Médio completo,NaN,NaN,NaN,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor privado,NaN,0 - 100
100337,18,Mulher,Parda,Médio incompleto,Sim,Pública,"Não, meu curso é online",Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100338,58,Homem,Preta,Fundamental incompleto,NaN,NaN,NaN,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,Conta própria,NaN,801 - 1.600
100339,47,Mulher,Parda,Fundamental incompleto,NaN,NaN,NaN,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
100340,83,Mulher,Parda,Sem instrução,NaN,NaN,NaN,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110204,74,Mulher,Branca,Fundamental incompleto,NaN,NaN,NaN,Não,Não,Não,...,Sim,Não,NaN,Sim,Negativo,Não,NaN,NaN,NaN,NaN
110205,50,Homem,Parda,Sem instrução,NaN,NaN,NaN,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110206,35,Mulher,Parda,Médio completo,NaN,NaN,NaN,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,"Trabalhador doméstico (empregado doméstico, cu...",NaN,301 - 600
110207,36,Homem,Parda,Fundamental incompleto,NaN,NaN,NaN,Não,Não,Não,...,Não,NaN,NaN,NaN,NaN,NaN,NaN,Empregado do setor público (inclusive empresas...,Municipal,1.601 - 3.000
